# 1. Preprocessing 

## 1.1 Requirements

In [2]:
pip3 install pandas
pip install --upgrade nltk
pip instaç stanza

## 1.2 Imports

In [1]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
#import stanza
#stanza.download('pt')

[nltk_data] Downloading package stopwords to /home/alina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-03-01 17:17:45 INFO: Downloading default packages for language: pt (Portuguese)...
2022-03-01 17:17:46 INFO: File exists: /home/alina/stanza_resources/pt/default.zip.
2022-03-01 17:17:50 INFO: Finished downloading models and saved to /home/alina/stanza_resources.


In [2]:
# Table only with news 'headline' and 'content'

data = pd.read_csv ('data.csv')
data_text = data.loc[:, ['headline', 'content']]

In [3]:
# Search for null values

for col in data_text.columns:
    print(col, data_text[col].isnull().sum())

headline 11
content 0


In [4]:
# Join 'headline' and 'content'

for index, row in data_text.iterrows():
    if isinstance(row['headline'], float): pass
    else: 
        if row['headline'] not in row['content']: row['content'] = row['headline'] + ' ' + row['content']
data_text.drop('headline', axis = 1, inplace = True)
pd.set_option('max_colwidth',150)
data_text

,content
0,"Prisão perpétua para homem que tentou assassinar senadora No dia 8 de Janeiro de 2011 Jared Loughner tentou matar, sem sucesso, a então senadora G..."
1,"John Nash, matemático de Uma Mente Brilhante, morre em acidente John Nash, o matemático e Nobel da Economia que foi retratado por Russell Crowe no..."
2,O mito da reeleição mínima garantida Cavaco sairá desta campanha pior do que entrou. Os casos das acções na SLN ou da casa de férias no Algarve nã...
3,"Morreu Rita Levi-Montalcini, a grande dama da ciência italiana A cientista e senadora italiana Rita Levi-Montalcini, Prémio Nobel da Medicina de 1..."
4,"Por trás da porta amarela, há um homem com problemas psicológicos Pub Ninguém sabe o que ele está a fazer. Ninguém sabe em que é que pensa, o que..."
...,...
654,Ao terceiro dia Hendrix ressuscitou Festival de Curtas Metragens recordou último concerto do guitarrista em Inglaterra Festival de Curtas Metrage...
655,Capital acordou tarde e tensa votação Jornada eleitoral decorreu com civismo e ansiedade Escaramuças fazem um morto e um ferido Jornada eleitoral...
656,"- out3022 Arguidos começam hoje a discutir as provas casa pia Rui Teixeira abandona processo de pedofilia, que será sorteado a outro magistrado..."
657,"""Nunca me deixei ser empurrado"" Alexandra Marques Ciente de que só passará à segunda volta ""se uma grande parte do PS"" votar nele, diz que Sócra..."


In [5]:
# Cleaning

irrelevant = ["pub", "pág", "reportagem", "clique", "subscrever", "partilhar", "dn", "publico.pt", "rtp", "cartaz", "tv", "hoje", "amanhã", "ontem", "segundafeira", "terçafeira", "quartafeira", "quintafeira", "sextafeira", "sábado", "domingo", "janeiro", "fevereiro", "março", "abril", "maio", "junho", "julho", "agosto", "outubro", "novembro", "dezembro"] # list with irrelevant words/expressions do the news content

stopwords = nltk.corpus.stopwords.words('portuguese') # pt stopwords

#nlp = stanza.Pipeline('pt') # pt stanza

def clean_text(text):

    text = text.lower() # lowercase
    text = re.sub(r'http\S+', '', text) # URLs removal
    text = re.sub('\[.*?\]', '', text) # remove text in square brackets
    text = re.sub(r"\([^()]*\)", '', text) # remove text within parentheses
    text = re.sub(r'-(me|te|se|nos|vos|o|os|a|as|lo|los|la|las|lhe|lhes|lha|lhas|lho|lhos|no|na|nas|mo|ma|mos|mas|to|ta|tos|tas)', '', text) # remove pronouns at the end of the wo
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('[‘’“”«|»…]', '', text)
    text = re.sub('\w*\d\w*', '', text) # remove words containing numbers
    text = ' '.join([word for word in text.split() if word not in stopwords]) # remove stopwords
    text = re.sub(r'Pág.*por pág$', '', text) # remove irrelevant text
    text = ' '.join([word for word in text.split() if word not in irrelevant]) 
    #text = nlp(text)
    #text = ' '.join([word.lemma for sent in text.sentences for word in sent.words]) 

    return text

2022-03-01 17:17:57 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2022-03-01 17:17:57 INFO: Use device: cpu
2022-03-01 17:17:57 INFO: Loading: tokenize
2022-03-01 17:17:57 INFO: Loading: mwt
2022-03-01 17:17:57 INFO: Loading: pos
2022-03-01 17:17:57 INFO: Loading: lemma
2022-03-01 17:17:57 INFO: Loading: depparse
2022-03-01 17:17:57 INFO: Done loading processors!


In [6]:
data_clean = pd.DataFrame(data_text.content.apply(lambda x: clean_text(x)))
data_clean

In [19]:
data_clean.to_csv()